In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
sns.set_style("whitegrid")

BASE_DIR = os.path.dirname(os.getcwd())
DATA_RAW_DIR = os.path.join(BASE_DIR, 'data', 'raw')
DATA_PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')

In [4]:
try:
    df_attendance = pd.read_csv(os.path.join(DATA_RAW_DIR, 'attendance.csv'))
    df_games = pd.read_csv(os.path.join(DATA_RAW_DIR, 'games.csv'))
    df_standings = pd.read_csv(os.path.join(DATA_RAW_DIR, 'standings.csv'))
    
    print("Berhasil load semua data.")
except FileNotFoundError as e:
    print(f"Error: File tidak ditemukan. \n{e}")

Berhasil load semua data.


In [5]:
print("DATA ATTENDANCE")
display(df_attendance.head())


print("\nINFO DATASET")
df_attendance.info()

DATA ATTENDANCE


,team,team_name,year,total,home,away,week,weekly_attendance
0,Arizona,Cardinals,2000,893926,387475,506451,1,77434.0
1,Arizona,Cardinals,2000,893926,387475,506451,2,66009.0
2,Arizona,Cardinals,2000,893926,387475,506451,3,NaN
3,Arizona,Cardinals,2000,893926,387475,506451,4,71801.0
4,Arizona,Cardinals,2000,893926,387475,506451,5,66985.0



INFO DATASET
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10846 entries, 0 to 10845
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   team               10846 non-null  object 
 1   team_name          10846 non-null  object 
 2   year               10846 non-null  int64  
 3   total              10846 non-null  int64  
 4   home               10846 non-null  int64  
 5   away               10846 non-null  int64  
 6   week               10846 non-null  int64  
 7   weekly_attendance  10208 non-null  float64
dtypes: float64(1), int64(5), object(2)
memory usage: 678.0+ KB


### **Cek & Handling Missing Values**

In [6]:
# 1. Cek jumlah missing values per kolom
print("Jumlah Missing Values")
print(df_attendance.isnull().sum())

# 2. Lihat baris yang weekly_attendance-nya kosong
print("\nContoh Data Kosong")
display(df_attendance[df_attendance['weekly_attendance'].isnull()].head())

df_clean = df_attendance.dropna(subset=['weekly_attendance']).copy()

print(f"\nJumlah baris sebelum drop: {len(df_attendance)}")
print(f"Jumlah baris setelah drop: {len(df_clean)}")

Jumlah Missing Values
team                   0
team_name              0
year                   0
total                  0
home                   0
away                   0
week                   0
weekly_attendance    638
dtype: int64

Contoh Data Kosong


,team,team_name,year,total,home,away,week,weekly_attendance
2,Arizona,Cardinals,2000,893926,387475,506451,3,NaN
31,Atlanta,Falcons,2000,964579,422814,541765,15,NaN
47,Baltimore,Ravens,2000,1062373,551695,510678,14,NaN
54,Buffalo,Bills,2000,1098587,560695,537892,4,NaN
71,Carolina,Panthers,2000,1095192,583489,511703,4,NaN



Jumlah baris sebelum drop: 10846
Jumlah baris setelah drop: 10208


In [7]:
# 1. Buat kolom baru 'full_name'
df_clean['full_name'] = df_clean['team'] + ' ' + df_clean['team_name']

# 2. Cek semua nama tim unik yang ada
unique_teams = sorted(df_clean['full_name'].unique())

print("\nDaftar Semua Tim (Cek Duplikasi Kota)")
for team in unique_teams:
    print(team)


Daftar Semua Tim (Cek Duplikasi Kota)
Arizona Cardinals
Atlanta Falcons
Baltimore Ravens
Buffalo Bills
Carolina Panthers
Chicago Bears
Cincinnati Bengals
Cleveland Browns
Dallas Cowboys
Denver Broncos
Detroit Lions
Green Bay Packers
Houston Texans
Indianapolis Colts
Jacksonville Jaguars
Kansas City Chiefs
Los Angeles Chargers
Los Angeles Rams
Miami Dolphins
Minnesota Vikings
New England Patriots
New Orleans Saints
New York Giants
New York Jets
Oakland Raiders
Philadelphia Eagles
Pittsburgh Steelers
San Diego Chargers
San Francisco 49ers
Seattle Seahawks
St. Louis Rams
Tampa Bay Buccaneers
Tennessee Titans
Washington Redskins


### **Fixing & Saving Data**

In [14]:
# 1. PREPARE STANDINGS (Klasemen)
df_standings_clean = df_standings.copy()
df_standings_clean['full_name'] = df_standings_clean['team'] + ' ' + df_standings_clean['team_name']

# Mapping Nama Tim
team_mapping = {
    'St. Louis Rams': 'Los Angeles Rams',
    'San Diego Chargers': 'Los Angeles Chargers',
    'Oakland Raiders': 'Las Vegas Raiders',
    'Houston Oilers': 'Tennessee Titans',
    'Tennessee Oilers': 'Tennessee Titans'
}
df_standings_clean['team_clean'] = df_standings_clean['full_name'].replace(team_mapping)
cols_standings = ['year', 'team_clean', 'wins', 'loss', 'playoffs', 'sb_winner']
df_standings_final = df_standings_clean[cols_standings]


# 2. PREPARE GAMES (Detail Pertandingan)
df_games_clean = df_games.copy()

# A. FIX TYPE DATA
# Paksa kolom 'week' jadi angka. Jika ada teks aneh (misal 'SuperBowl'), ubah jadi NaN lalu buang/isi.
df_games_clean['week'] = pd.to_numeric(df_games_clean['week'], errors='coerce')
# Isi NaN dengan 0 atau drop (kita pilih drop yang kosong agar jadi integer bersih)
df_games_clean = df_games_clean.dropna(subset=['week'])
# Ubah jadi integer (bilangan bulat) agar sama dengan attendance
df_games_clean['week'] = df_games_clean['week'].astype(int)

# B. Standardisasi Nama Home Team
df_games_clean['home_full_name'] = df_games_clean['home_team_city'] + ' ' + df_games_clean['home_team_name']
df_games_clean['home_team_clean'] = df_games_clean['home_full_name'].replace(team_mapping)

# C. Hitung Home Score & Away Score
conditions = [
    df_games_clean['home_team'] == df_games_clean['winner'],
    df_games_clean['home_team'] != df_games_clean['winner']
]
choices_home = [df_games_clean['pts_win'], df_games_clean['pts_loss']]
choices_away = [df_games_clean['pts_loss'], df_games_clean['pts_win']]

df_games_clean['home_score'] = np.select(conditions, choices_home, default=df_games_clean['pts_win'])
df_games_clean['away_score'] = np.select(conditions, choices_away, default=df_games_clean['pts_loss'])

cols_games = ['year', 'week', 'home_team_clean', 'away_team', 'home_score', 'away_score', 'day']
df_games_final = df_games_clean[cols_games]


# 3. MERGING

# Step A: Gabung Attendance + Standings
df_step1 = pd.merge(
    df_clean, 
    df_standings_final, 
    on=['team_clean', 'year'], 
    how='left'
)

# Step B: Gabung Hasil Step A + Games
df_final = pd.merge(
    df_step1,
    df_games_final,
    left_on=['team_clean', 'year', 'week'],       
    right_on=['home_team_clean', 'year', 'week'], 
    how='left'
)

# 4. CLEANING AKHIR & SIMPAN
if 'home_team_clean' in df_final.columns:
    df_final.drop(columns=['home_team_clean'], inplace=True)

output_path = os.path.join(DATA_PROCESSED_DIR, 'nfl_attendance_complete.csv')
df_final.to_csv(output_path, index=False)

print(f"\n3 File berhasil digabung.")
print(f"Dimensi Final: {df_final.shape}")
display(df_final.head())


3 File berhasil digabung.
Dimensi Final: (10208, 18)


,team,team_name,year,total,home,away,week,weekly_attendance,full_name,team_clean,wins,loss,playoffs,sb_winner,away_team,home_score,away_score,day
0,Arizona,Cardinals,2000,893926,387475,506451,1,77434.0,Arizona Cardinals,Arizona Cardinals,3,13,No Playoffs,No Superbowl,NaN,NaN,NaN,NaN
1,Arizona,Cardinals,2000,893926,387475,506451,2,66009.0,Arizona Cardinals,Arizona Cardinals,3,13,No Playoffs,No Superbowl,Dallas Cowboys,32.0,31.0,Sun
2,Arizona,Cardinals,2000,893926,387475,506451,4,71801.0,Arizona Cardinals,Arizona Cardinals,3,13,No Playoffs,No Superbowl,Green Bay Packers,3.0,29.0,Sun
3,Arizona,Cardinals,2000,893926,387475,506451,5,66985.0,Arizona Cardinals,Arizona Cardinals,3,13,No Playoffs,No Superbowl,NaN,NaN,NaN,NaN
4,Arizona,Cardinals,2000,893926,387475,506451,6,44296.0,Arizona Cardinals,Arizona Cardinals,3,13,No Playoffs,No Superbowl,Cleveland Browns,29.0,21.0,Sun
